<a href="https://colab.research.google.com/github/pvrancx/torch_isr/blob/master/colabs/evaluate_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install pytorch-lightning

Checkount repo

In [0]:
% cd "/content"
! git clone "https://github.com/pvrancx/torch_isr.git"
% cd "/content/torch_isr"
! git pull

In [0]:
from pytorch_lightning import Trainer
import torch
from torch.utils.data import random_split, DataLoader
from torchvision.transforms import transforms
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from isr.datasets.bsd import load_bsd300
from isr.datasets.transforms import ChannelSelect
from isr.datasets.isr import IsrDataset
from isr.models.srcnn import SrCnn, SubPixelSrCnn
from isr.models.srresnet import SrResNet
from isr.models.srgan import SrGan, Discriminator

from super_resolve import super_resolve_ycbcr, super_resolve_rgb

import os
from PIL import Image
from argparse import ArgumentParser
import matplotlib.pyplot as plt
%matplotlib inline

Load previously trained model


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
model = SrResNet.load_from_checkpoint('/content/torch_isr/trained_models/srresnet4x.ckpt')

Load test data

In [0]:
bsd300_test = load_bsd300('../data', split='test', image_mode='RGB')
test_data = IsrDataset(
        bsd300_test,
        output_size=None,
        scale_factor=4,
        deterministic=True
)

Apply model

In [0]:
img_idx = 23

lr_img, hr_img = test_data[img_idx]
scale_factor = model.scale_factor
interpolate = lr_img.resize((lr_img.size[0] * scale_factor, lr_img.size[1] * scale_factor), Image.BICUBIC)
output = super_resolve_rgb(model, lr_img)

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15,15))
ax1.imshow(lr_img)
ax1.set_title('low res')
ax1.set_axis_off()
ax2.imshow(interpolate)
ax2.set_title('bicubic')
ax2.set_axis_off()
ax3.imshow(output)
ax3.set_title('srresnet')
ax3.set_axis_off()
ax4.imshow(hr_img)
ax4.set_title('original')
ax4.set_axis_off()